## How many and which types of accounts have created datasets in Root each day?

### Create and check filtered dataframe

In [ ]:
# Create new dataframe

countbyAccountType = (
    rawDataDF.query(
        'deposit_parent_dataverse == "1"\
        & deposit_type == "Dataset"\
        & deposit_createdate > "2018-10-31"\
        & deposit_createdate < "2020-11-01"')
    
    # Retain only these columns
    [['user_account_id_of_deposit', 'deposit_createdate', 'user_account_type']]
    
    .groupby(['deposit_createdate', 'user_account_type', 'user_account_id_of_deposit']).count()
    .sort_values(by=['deposit_createdate'], ascending=True)
    .rename(columns={'user_account_id_of_deposit': 'count_of_accounts'})
)

# Reset dataframe index (so that values in deposit_createdate repeat)
countbyAccountType.reset_index(drop=False, inplace=True)

countbyAccountType = (
    countbyAccountType.groupby(['deposit_createdate', 'user_account_type']).count()
    .rename(columns={'user_account_id_of_deposit': 'count_of_accounts'})
)

# Reset dataframe index (so that values in deposit_createdate repeat)
countbyAccountType.reset_index(drop=False, inplace=True)


In [ ]:
print(countbyAccountType.shape)
countbyAccountType.head(5)


## How often are spam and test dataverses published?

```
select
    to_char(starttime, 'YYYY-MM-DD') as dataverse_publish_date,
    useridentifier as depositor_id,
    substring(info from '(?<=\:\[)\d*')::int as dataverse_id
from actionlogrecord
where actionsubtype ilike '%PublishDataverseCommand%' and actionresult = 'OK'
    and useridentifier not in ('@merce', '@dataverseAdmin', '@sonia', '@Dwayne', '@juliangautier', '@kmika')
    and substring(info from '(?<=\:\[)\d*')::int in(
        select 
            substring(info from '(?<=doomed\:\[)\d*')::int
        from actionlogrecord
        where
            actionsubtype ilike '%DeleteDataverseCommand%'
            and useridentifier in ('@merce', '@dataverseAdmin', '@sonia', '@Dwayne', '@juliangautier', '@kmika')
            and info ilike 'owner:[1 Harvard Dataverse] doomed%'
            and starttime > '2018-10-31'
            and actionresult = 'OK'
    )
```